In [113]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import robin_stocks.robinhood as r

### Important Links:
https://robin-stocks.readthedocs.io/en/latest/robinhood.html

https://github.com/jmfernandes/robin_stocks/blob/master/Robinhood.rst

In [114]:
class Stocks:
    def __init__(self, u="", p="", mfa=""):
        self.username = u
        self.password = p
        self.mfa = mfa
        self.login = r.login(username, password, mfa_code=MFA)
        self.update_stats()
    
    
    def update_stats(self):
        self.positions = r.get_open_stock_positions()
        self.tickers = [r.get_symbol_by_url(item["instrument"]) for item in self.positions]
        self.tickerInfo =  r.get_quotes(tickers)
        self.quantities = [float(item["quantity"]) for item in self.positions]
        self.prevClose = r.get_quotes(tickers, "previous_close")
        self.lastPrice = r.get_quotes(tickers, "last_trade_price")
        self.lastExtPrice = r.get_quotes(tickers, "last_trade_price")
        self.boughtAt = [float(item["average_buy_price"]) for item in self.positions]
        self.bidPrice = [float(item["bid_price"]) for item in self.tickerInfo]
        self.askPrice = [float(item["ask_price"]) for item in self.tickerInfo]
        
        self.update_daily_stats()
        self.update_total_stats()


        
    def update_daily_stats(self):
        #Daily calculations
        self.dailyProfitPerShare = [float(self.lastPrice[i]) - float(self.prevClose[i]) for i in range(len(self.tickers))]
        self.dailyPercentChange = [ 100.0 * self.dailyProfitPerShare[i] / float(self.prevClose[i]) for i in range(len(self.tickers)) ]
        self.dailyProfit = [self.dailyProfitPerShare[i] * self.quantities[i] for i in range(len(self.tickers))]
    
    def update_total_stats(self):
        self.totalProfitPerShare = [float(self.lastPrice[i]) - float(self.boughtAt[i]) for i in range(len(tickers))]
        self.totalPercentChange = [ 100.0 * self.totalProfitPerShare[i] / float(self.boughtAt[i]) for i in range(len(self.tickers)) ]
        self.totalProfit = [self.totalProfitPerShare[i] * self.quantities[i] for i in range(len(self.tickers))]

        
    def printDailyInfo(self):
        self.dailyTickersPerf = list(zip(self.dailyProfit, self.dailyPercentChange, self.tickers))
        self.dailyTickersPerf.sort(reverse=True)
        print ("My Positions Performance:")
        print ("Ticker | DailyGain | PercentChange")
        for item in self.dailyTickersPerf:
          print ("%s %f$ %f%%" % (item[2], item[0], item[1]))

        print ("Net Gain:", sum(self.dailyProfit))
    
    def printTotalInfo(self):
        self.totalTickersPerf = list(zip(self.totalProfit, self.totalPercentChange, self.tickers))
        self.totalTickersPerf.sort(reverse=True)
        print ("My Positions Performance:")
        print ("Ticker | DailyGain | PercentChange")
        for item in self.totalTickersPerf:
          print ("%s %f$ %f%%" % (item[2], item[0], item[1]))

        print ("Net Gain:", sum(self.totalProfit))    
        
        
    
    def logout(self):
        r.logout()

In [115]:
#Pull user name in form .env
# import * from credentials.env
import os
from dotenv import load_dotenv
import pyotp

load_dotenv()

code = os.getenv('MFA')

MFA = pyotp.TOTP(code).now()
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

In [116]:
a = Stocks(username, password, MFA)

In [117]:
#a.printDailyInfo()

In [118]:
#a.printTotalInfo()

In [119]:
a.logout()